In [19]:
task = "readmission"

In [20]:
import json

if task == "mortality" or task == "readmission":
    with open("../../clustering/ccscm_ccsproc_atc3/clusters_th015.json", "r") as f:
        clusters = json.load(f)
elif task == "lenofstay" or task == "drugrec":
    with open("../../clustering/ccscm_ccsproc/clusters_th015.json", "r") as f:
        clusters = json.load(f)

In [21]:
import numpy as np

def cosine_similarity(u, v):
    """
    计算两个向量的余弦相似度，安全处理零向量情况
    """
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    
    # 如果任一向量为零向量，返回0相似度
    if norm_u == 0 or norm_v == 0:
        return 0.0
    
    return np.dot(u, v) / (norm_u * norm_v)

In [22]:
from get_emb import embedding_retriever

terms = None
if task == "mortality":
    terms = ['death', 'mortality', 'cause death', 'lead to death', 'high risk', "deadly"]
elif task == "readmission":
    terms = ['rehospitalization', 'readmission']
elif task == "lenofstay":
    terms = ["length of stay'", "bed days", "time in hospital"]
elif task == "drugrec":
    terms = ["drug recommendation", "prescription", "drug", "medication", "treatment"]

term_embs = []

for term in terms:
    term_embs.append(embedding_retriever(term))

tmp = {}

for clus in clusters.keys():
    tmp[clus] = 0
    for term_emb in term_embs:
        tmp[clus] += cosine_similarity(clusters[clus]['embedding'], term_emb)   


In [23]:
float(tmp['0'])

/tmp/ipykernel_6035/3265352447.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  float(tmp['0'])


1.0039951139368974

In [24]:
min_similarity = min(tmp.values())
max_similarity = max(tmp.values())

for clus in clusters.keys():
    tmp[clus] = (float(tmp[clus]) - min_similarity) / (max_similarity - min_similarity)


/tmp/ipykernel_6035/3433933721.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tmp[clus] = (float(tmp[clus]) - min_similarity) / (max_similarity - min_similarity)


In [25]:
min_similarity

array([0.4634868])

In [26]:
for clus in clusters.keys():
    tmp[clus] = float(tmp[clus] ** 3)


/tmp/ipykernel_6035/704129713.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tmp[clus] = float(tmp[clus] ** 3)


In [27]:
max(tmp.values())

1.0

In [28]:
for clus in clusters.keys():
    if task == "mortality":
        clusters[clus]['attention_mortality'] = tmp[clus]
    elif task == "readmission":
        clusters[clus]['attention_readmission'] = tmp[clus]
    elif task == "lenofstay":
        clusters[clus]['attention_lenofstay'] = tmp[clus]
    elif task == "drugrec":
        clusters[clus]['attention_drugrec'] = tmp[clus]

In [29]:

if task == "mortality" or task == "readmission":

    with open("../../clustering/ccscm_ccsproc_atc3/clusters_th015.json", "w") as f:
        json.dump(clusters, f)

elif task == "lenofstay" or task == "drugrec":
    
    with open("../../clustering/ccscm_ccsproc/clusters_th015.json", "w") as f:
        json.dump(clusters, f)

In [30]:
import pickle
import numpy as np

if task == "mortality" or task == "readmission":
    attn_file = f"../../data/ccscm_ccsproc_atc3/attention_weights_{task}.pkl"
elif task == "lenofstay" or task == "drugrec":
    attn_file = f"../../data/ccscm_ccsproc/attention_weights_{task}.pkl"
attn = np.ndarray(shape=(len(clusters), 1))

for i in range(len(clusters)):
    idx = str(i)
    attn[i] = clusters[idx][f'attention_{task}']

with open(attn_file, "wb") as f:
    pickle.dump(attn, f)
